# <strong> Hello World! </strong> 


In [1]:
# Open and display the script:
f = open("Hello_world.py", "r")
print(f.read())
f.close()


from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

print("Hello world from rank %r!" % rank)



In [20]:
# Parallel run the script in terminal

# ! -- running the command in terminal
# mpirun -- command for running scripts with MPI
# -n <Number> -- specify the number of processes

!mpiexec -n 2 python Hello_world.py

Invalid MIT-MAGIC-COOKIE-1 key--------------------------------------------------------------------------
mpiexec was unable to find the specified executable file, and therefore
did not launch the job.  This error was first reported for process
rank 0; it may have occurred for other processes as well.

NOTE: A common cause for this error is misspelling a mpiexec command
      line parameter option (remember that mpiexec interprets the first
      unrecognized command line token as the executable).

Node:       ildar-machine
Executable: python
--------------------------------------------------------------------------
2 total processes failed to start


In [3]:
# Use this when in Google colab
!mpiexec --allow-run-as-root -n 10 python -m mpi4py Hello_world.py

Invalid MIT-MAGIC-COOKIE-1 key--------------------------------------------------------------------------
There are not enough slots available in the system to satisfy the 10
slots that were requested by the application:

  python

Either request fewer slots for your application, or make more slots
available for use.

A "slot" is the Open MPI term for an allocatable unit where we can
launch a process.  The number of slots available are defined by the
environment in which Open MPI processes are run:

  1. Hostfile, via "slots=N" clauses (N defaults to number of
     processor cores if not provided)
  2. The --host command line parameter, via a ":N" suffix on the
     hostname (N defaults to 1 if not provided)
  3. Resource manager (e.g., SLURM, PBS/Torque, LSF, etc.)
  4. If none of a hostfile, the --host command line parameter, or an
     RM is present, Open MPI defaults to the number of processor cores

In all the above cases, if you want Open MPI to default to the number
of hardware t

# <strong> Blocking point-to-point communication
</strong> 


In [4]:
f = open("Simple_blocking_p2p.py", "r")
print(f.read())
f.close()


from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = { 'a': 7, 'b': 3.14 }
    comm.send(data, dest = 1, tag = 11)

    print("rank = %r, data = %r" % (rank, data))

elif rank == 1:
    data = comm.recv(source = 0, tag = 11)

    print("rank = %r, data = %r" % (rank, data))

else:
    print("rank = %r, data = <NONE>" % rank)



In [5]:
!mpiexec.exe -n 3 python Simple_blocking_p2p.py

/bin/bash: mpiexec.exe: command not found


# <strong> NumPy blocking point-to-point communication </strong> 


In [6]:
f = open("Numpy_blocking_p2p.py", "r")
print(f.read())
f.close()


from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# In real code this section might read in data from file
if rank == 0:
    size = 10
    comm.send(size, dest = 1)

    data = np.linspace(0.0, 3.14, size)
    comm.Send(data, dest = 1)

    print("Sended data = %r" % data)

elif rank == 1:
    size = comm.recv(source = 0)

    # Allocate space to receive the array
    data = np.empty(size, dtype = 'd')
    comm.Recv(data, source = 0)

    print("Received data = %r" % data)

else:
    data = None

    print("Other data = %r" % data)



In [7]:
!mpirun -n 3 python Numpy_blocking_p2p.py

Invalid MIT-MAGIC-COOKIE-1 key--------------------------------------------------------------------------
mpirun was unable to find the specified executable file, and therefore
did not launch the job.  This error was first reported for process
rank 0; it may have occurred for other processes as well.

NOTE: A common cause for this error is misspelling a mpirun command
      line parameter option (remember that mpirun interprets the first
      unrecognized command line token as the executable).

Node:       ildar-machine
Executable: python
--------------------------------------------------------------------------
3 total processes failed to start


# <strong> Non-blocking point-to-point communication
</strong> 


In [8]:
f = open("Nonblocking_p2p.py", "r")
print(f.read())
f.close()


from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

data = rank

print("rank = %r, data = %r" % (rank, data))

# All processes wait here for all
comm.Barrier()
print("Barrier")
# The same here
# Try to comment one of and both of barriers
comm.Barrier()

# Send, not wait
request = comm.isend(data, dest = (rank + 1) % size)

# Receive, not wait
request = comm.irecv(source = (rank - 1) % size)
# Wait for corresponding sender
data = request.wait()

print("rank = %r, data = %r" % (rank, data))



In [9]:
!mpirun -n 3 python Nonblocking_p2p.py

Invalid MIT-MAGIC-COOKIE-1 key--------------------------------------------------------------------------
mpirun was unable to find the specified executable file, and therefore
did not launch the job.  This error was first reported for process
rank 0; it may have occurred for other processes as well.

NOTE: A common cause for this error is misspelling a mpirun command
      line parameter option (remember that mpirun interprets the first
      unrecognized command line token as the executable).

Node:       ildar-machine
Executable: python
--------------------------------------------------------------------------
3 total processes failed to start


# <strong> Collective communication: Sum-reduce </strong>

In [10]:
f = open("Sum_reduce.py", "r")
print(f.read())
f.close()


from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# Python objects sum-reduce
result = comm.reduce(rank, op = MPI.SUM, root = 0)

if rank == 0:
    print("result = %r" % result)

# Numpy objects sum-reduce
sendbuf = np.empty(2, dtype = 'i')
sendbuf[0] = 1
sendbuf[1] = rank

if rank == 0:
    recvbuf = np.empty(2, dtype = 'i')
else:
    recvbuf = None

comm.Reduce(sendbuf, recvbuf, op = MPI.SUM, root = 0)

if rank == 0:
    print("recvbuf = %r" % recvbuf)



In [11]:
!mpirun -n 5 python Sum_reduce.py

Invalid MIT-MAGIC-COOKIE-1 key--------------------------------------------------------------------------
mpirun was unable to find the specified executable file, and therefore
did not launch the job.  This error was first reported for process
rank 0; it may have occurred for other processes as well.

NOTE: A common cause for this error is misspelling a mpirun command
      line parameter option (remember that mpirun interprets the first
      unrecognized command line token as the executable).

Node:       ildar-machine
Executable: python
--------------------------------------------------------------------------
5 total processes failed to start


# <strong> Collective communication: Broadcast </strong>

In [12]:
f = open("Broadcast.py", "r")
print(f.read())
f.close()


from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

data = rank * rank

if rank == 0:
    data = { 'key1' : [7, 2.72, 2 + 3j], 'key2' : ('abc', 'xyz') }
else:
    data = None

print("rank = %r: data = %r" % (rank, data))

comm.Barrier()
data = comm.bcast(data, root=0)

print("rank = %r: data = %r" % (rank, data))



In [13]:
!mpirun -n 5 python Broadcast.py

Invalid MIT-MAGIC-COOKIE-1 key--------------------------------------------------------------------------
mpirun was unable to find the specified executable file, and therefore
did not launch the job.  This error was first reported for process
rank 0; it may have occurred for other processes as well.

NOTE: A common cause for this error is misspelling a mpirun command
      line parameter option (remember that mpirun interprets the first
      unrecognized command line token as the executable).

Node:       ildar-machine
Executable: python
--------------------------------------------------------------------------
5 total processes failed to start


# <strong> Collective communication: Scatter </strong>

In [14]:
f = open("Scatter.py", "r")
print(f.read())
f.close()


from mpi4py import MPI
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

print('The number of processor is %s' % sys.argv[1])

if rank == 0:
    data = [x**2 for x in range(comm.Get_size())]
else:
    data = None

print("[Before] rank = %r: data = %r" % (rank, data))

data = comm.scatter(data, root = 0)

print("[After] rank = %r: data = %r" % (rank, data))



In [15]:
!mpirun -n 5 python Scatter.py

Invalid MIT-MAGIC-COOKIE-1 key--------------------------------------------------------------------------
mpirun was unable to find the specified executable file, and therefore
did not launch the job.  This error was first reported for process
rank 0; it may have occurred for other processes as well.

NOTE: A common cause for this error is misspelling a mpirun command
      line parameter option (remember that mpirun interprets the first
      unrecognized command line token as the executable).

Node:       ildar-machine
Executable: python
--------------------------------------------------------------------------
5 total processes failed to start


# <strong> Collective communication: Gather </strong>

In [16]:
f = open("Gather.py", "r")
print(f.read())
f.close()


from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

data = rank * rank

print("[Before] rank = %r: data = %r" % (rank, data))

data = comm.gather(data, root = 0)

print("[After] rank = %r: data = %r" % (rank, data))



In [17]:
!mpirun -n 5 python Gather.py

Invalid MIT-MAGIC-COOKIE-1 key--------------------------------------------------------------------------
mpirun was unable to find the specified executable file, and therefore
did not launch the job.  This error was first reported for process
rank 0; it may have occurred for other processes as well.

NOTE: A common cause for this error is misspelling a mpirun command
      line parameter option (remember that mpirun interprets the first
      unrecognized command line token as the executable).

Node:       ildar-machine
Executable: python
--------------------------------------------------------------------------
5 total processes failed to start
